In [1]:
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
pipmain(['install','gekko'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


0

In [13]:
from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt

#%% Simulation

s = GEKKO(name='cstr-sim')

#1 step of simulation, discretization matches MHE
s.time = np.linspace(0,.1,2)

#Receive measurement from simulated control
F = s.MV(value=100,name='F')
F.FSTATUS = 1 #receive measurement
F.STATUS = 0  #don't optimize

Qk = s.MV(value=10,name='Qk')
Qk.FSTATUS = 1 #receive measurement
Qk.STATUS = 0  #don't optimize

#State variables to watch
CA = s.SV(value=5.1, ub=5.1, lb=0,name='CA')
CB = s.SV(value=1.5, ub=1.5, lb=0,name='CB')
T = s.SV(value=100,ub=500,lb=50,name='T')
Tk = s.SV(value=110,ub=500,lb=50,name='Tk')

#other parameters
V = s.Param(value=10)
rho = s.Param(value=0.9342)
k10= s.Param(value=1.287E12) # in h-1
k20= s.Param(value=1.287E12) # in h-1
k30= s.Param(value=9.043E9) # in L/molA.h
mE1R= s.Param(value=-9758.3) # in K
mE2R= s.Param(value=-9758.3) # in K
mE3R= s.Param(value=-8560.0) # in K
mdeltaHAB= s.Param(value=-4.20) # in kJ/molA
mdeltaHBC= s.Param(value=11.0) # in kJ/molB
mdeltaHAD= s.Param(value=41.85) # in kJ/molA
Cp = s.Param(value=3.01) #heat capacity (kJ/kg K)
delta_CAin = s.Param(value=1.0)
Tin = s.Param(value=135)
Ar = s.Param(value=0.215) #jacket area (m2)
Kw = s.Param(value=4032) #jacket heat transfer coefficient (kJ/h m2 K)
Cp_k = s.Param(value=2.0) # Coolant heat capacity [kj/kg.k]
A_R = s.Param(value=0.215) # Area of reactor wall [m^2]
m_k = s.Param(value=5.0) # Coolant mass[kg]

#Variables
rA = s.Var()
rB = s.Var()
rA2 = s.Var()

#Rate equations
s.Equation(rA == k10 * s.exp(mE1R/(T+273.15))*CA)
s.Equation(rB == k20 * s.exp(mE2R/(T+273.15))*CB)
s.Equation(rA2 == k30 * s.exp(mE3R/(T+273.15))*CA**2)

#CSTR equations
s.Equation(CA.dt() == F*(delta_CAin-CA) - rA - rA2)
s.Equation(CB.dt() == -F*CB + rA - rB)
s.Equation(T.dt() == F*(Tin - T) + 1/rho/Cp*(rA*mdeltaHAB + rB*mdeltaHBC + rA2*mdeltaHAD) + Kw*Ar/(rho*Cp*V)*(Tk-T))
s.Equation(Tk.dt() == (Qk + (T - Tk))/(m_k*Cp_k))

#Options
s.options.IMODE = 4 #dynamic simulation
s.options.NODES = 3
s.options.SOLVER = 3



In [14]:
#%% MHE

#Model

m = GEKKO(name='cstr-mhe')

#6 time points in horizon
m.time = np.linspace(0,.5,6)

#Parameter to Estimate
delta_CAin_mhe = m.FV(value=1.0,name='delta_CAin')
delta_CAin_mhe.STATUS = 1 #estimate
delta_CAin_mhe.FSTATUS = 0 #no measurements

k10_mhe = m.FV(value=1.287E12,name='k10')
k10_mhe.STATUS = 1 #estimate
k10_mhe.FSTATUS = 0 #no measurements

Cp_mhe = m.FV(value=3.01,name='Cp')
Cp_mhe.STATUS = 1 #estimate
Cp_mhe.FSTATUS = 0 #no measurements

#upper and lower bounds for optimizer
delta_CAin_mhe.LOWER = 0
delta_CAin_mhe.UPPER = 10

k10_mhe.LOWER = 1.0E12
k10_mhe.UPPER = 1.5E12

Cp_mhe.LOWER = 1
Cp_mhe.UPPER = 5

#Measurement input
F_mhe = m.MV(value=100,name='F')
F_mhe.FSTATUS = 1 #receive measurement
F_mhe.STATUS = 0  #don't estimate

Qk_mhe = m.MV(value=10,name='Qk')
Qk_mhe.FSTATUS = 1 #receive measurement
Qk_mhe.STATUS = 0  #don't estimate

#Measurement to match simulation with
CB_mhe = m.CV(value=1.5, ub=1.5, lb=0,name='CB')
CB_mhe.STATUS = 1  #minimize error between simulation and measurement
CB_mhe.FSTATUS = 1 #receive measurement
CB_mhe.MEAS_GAP = 0.1 #measurement deadband gap

T_mhe = m.CV(value=100,ub=500,lb=50,name='T')
T_mhe.STATUS = 1  #minimize error between simulation and measurement
T_mhe.FSTATUS = 1 #receive measurement
T_mhe.MEAS_GAP = 0.1 #measurement deadband gap

#State to watch
CA_mhe = m.SV(value=5.1, ub=5.1, lb=0,name='CA')
Tk_mhe = m.SV(value=110,ub=500,lb=50,name='Tk')

#Other parameters
V = m.Param(value=10)
rho = m.Param(value=0.9342)
k10= m.Param(value=1.287E12) # in h-1
k20= m.Param(value=1.287E12) # in h-1
k30= m.Param(value=9.043E9) # in L/molA.h
mE1R= m.Param(value=-9758.3) # in K
mE2R= m.Param(value=-9758.3) # in K
mE3R= m.Param(value=-8560.0) # in K
mdeltaHAB= m.Param(value=-4.20) # in kJ/molA
mdeltaHBC= m.Param(value=11.0) # in kJ/molB
mdeltaHAD= m.Param(value=41.85) # in kJ/molA
Cp = m.Param(value=3.01) #heat capacity (kJ/kg K)
delta_CAin = m.Param(value=1.0)
Tin = m.Param(value=135)
Ar = m.Param(value=0.215) #jacket area (m2)
Kw = m.Param(value=4032) #jacket heat transfer coefficient (kJ/h m2 K)
Cp_k = m.Param(value=2.0) # Coolant heat capacity [kj/kg.k]
A_R = m.Param(value=0.215) # Area of reactor wall [m^2]
m_k = m.Param(value=5.0) # Coolant mass[kg]

#Equation variables(2 other DOF from CV and FV)
rA = s.Var()
rB = s.Var()
rA2 = s.Var()

#Rate equations
m.Equation(rA == k10 * s.exp(mE1R/(T+273.15))*CA)
m.Equation(rB == k20 * s.exp(mE2R/(T+273.15))*CB)
m.Equation(rA2 == k30 * s.exp(mE3R/(T+273.15))*CA**2)

#CSTR equations
m.Equation(CA.dt() == F*(delta_CAin-CA) - rA - rA2)
m.Equation(CB.dt() == -F*CB + rA - rB)
m.Equation(T.dt() == F*(Tin - T) + 1/rho/Cp*(rA*mdeltaHAB + rB*mdeltaHBC + rA2*mdeltaHAD) + Kw*Ar/(rho*Cp*V)*(Tk-T))
m.Equation(Tk.dt() == (Qk + (T - Tk))/(m_k*Cp_k))

#Global Tuning
m.options.IMODE = 5 #MHE
m.options.EV_TYPE = 1
m.options.NODES = 3
m.options.SOLVER = 3 #IPOPT

#%% Loop

# number of cycles to run
cycles = 50

# step in CA_in at cycle 6
CAin_meas = np.empty(cycles)
CAin_meas[0:15] = 5.1
CAin_meas[5:cycles] = 4.1
dt = 0.1 # min
time = np.linspace(0,cycles*dt-dt,cycles) # time points for plot

# allocate storage
delta_CAin_meas = np.empty(cycles)
CA_meas = np.empty(cycles)
CB_meas = np.empty(cycles)
T_meas = np.empty(cycles)
F_meas = np.empty(cycles)
Qk_meas = np.empty(cycles)

delta_CAin_mhe_store = np.empty(cycles)
k10_mhe_store = np.empty(cycles)
Cp_mhe_store = np.empty(cycles)

CA_mhe_store = np.empty(cycles)
CB_mhe_store = np.empty(cycles)
T_mhe_store = np.empty(cycles)


for i in range(cycles):

    ## Process
    # inputs F/V and Qk/kw*Ar
    F.MEAS = F_meas[i]
    Qk.MEAS = Qk_meas[i]

    # simulate process model, 1 time step
    s.solve()

    # retrieve delta_CAin, Cb and T measurements from the process
    delta_CAin_meas[i] = CA.MODEL
    CB_meas[i] = CB.MODEL
    T_meas[i] = T.MODEL

    ## Estimator
    # input process measurements
    # inputs F/V and Qk/kw*Ar
    F_mhe.MEAS = F_meas[i]
    Qk_mhe.MEAS = Qk_meas[i]

    # input T (reactor temperature)
    T_mhe.MEAS = T_meas[i] #CV

    # solve process model, 1 time step
    m.solve()

    # check if successful
    if m.options.APPSTATUS == 1:
        # retrieve solution
        delta_CAin_mhe_store[i] = delta_CAin_mhe.NEWVAL
        k10_mhe_store[i] = k10_mhe.NEWVAL
        Cp_mhe_store[i] = Cp_mhe.NEWVAL
        CA_mhe_store[i] = CA_mhe.MODEL
        CB_mhe_store[i] = CB_mhe.MODEL
        T_mhe_store[i] = T_mhe.MODEL
    else:
        # failed solution
        delta_CAin_mhe_store[i] = 0
        k10_mhe_store[i] = 0
        Cp_mhe_store[i] = 0
        CA_mhe_store[i] = 0
        CB_mhe_store[i] = 0
        T_mhe_store[i] = 0

    print('MHE results: delta_Cain (estimated)=' + str(delta_CAin_mhe_store[i]) + \
        ' delta_Cain (actual)=' + str(delta_Cain_meas[i]) + \
        ' k10 (estimated)=' + str(k10_mhe_store[i]) + \
        ' k10 (actual)=1.287e12' + \
        ' Cp (estimated)=' + str(Cp_mhe_store[i]) + \
        ' Cp (actual)=3.01')

#%% plot results
plt.figure(figsize=(20,10))
plt.subplot(4,2,1)
plt.plot(time,F_meas,'k-',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('Flow F/V (1/h)')
plt.legend('F/V')

plt.subplot(4,2,2)
plt.plot(time,Qk_meas,'k-',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('Heat removed from jacket Qk/kw*Ar (K)')
plt.legend('Qk/kw*Ar')

plt.subplot(4,2,3)
plt.plot(time,delta_Cain_meas,'k--',linewidth=2)
plt.plot(time,delta_CAin_mhe_store,'r:',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('delta_CAin')
plt.legend(['Actual delta_CAin','Predicted delta_CAin'],loc='best')

plt.subplot(4,2,4)
plt.plot([0,time[-1]],[1.287e12,1.287e12],'k--')
plt.plot(time,k10_mhe_store,'r:',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('k10')
plt.legend(['Actual k10','Predicted k10'],loc='best')

plt.subplot(4,2,5)
plt.plot([0,time[-1]],[3.01,3.01],'k--')
plt.plot(time,Cp_mhe_store,'r:',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('Cp')
plt.legend(['Actual Cp','Predicted Cp'],loc='best')

plt.subplot(4,2,6)
plt.plot(time,T_meas,'ro')
plt.plot(time,T_mhe_store,'b-',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('Reactor T (K)')
plt.legend(['Measured T','Predicted T'],loc='best')

plt.subplot(4,2,7)
plt.plot(time,CB_meas,'go')
plt.plot(time,CB_mhe_store,'m-',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('Reactor C_b (mol/L)')
plt.legend(['Measured C_b','Predicted C_b'],loc='best')
plt.show()

plt.subplot(4,2,8)
plt.plot(time,CA_meas,'go')
plt.plot(time,CA_mhe_store,'m-',linewidth=2)
plt.axis([0,time[-1]])
plt.ylabel('Reactor C_a (mol/L)')
plt.legend(['Measured C_a','Predicted C_a'],loc='best')
plt.show()

apm 35.229.243.198_cstr-sim <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           31
   Intermediates:            0
   Connections  :            0
   Equations    :            7
   Residuals    :            7
 
 Number of state variables:             30
 Number of total equations: -           24
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              6
 
 @error: Degrees of Freedom
 * Error: DOF must be zero for this mode
 STOPPING...


Exception: ignored